In [1]:
import pandas as pd
test_set = pd.read_excel("../input/summerization/Competition CERIST -- Summarization/Dataset 2 Arabic + English (XL_sum)/Arabic/dataset_XL_sum_v1.0_test_ar.xlsx")


test_set['Summary'] = test_set['Summary'].replace(r'\n', '', regex = True)
test_set['Document'] = test_set['Document'].replace(r'\n', '', regex = True)

print(test_set.shape)

(4687, 2)


In [2]:
from datasets import Dataset
test_ds = Dataset.from_pandas(test_set)


In [3]:
!pip install transformers pyarabic
!git clone https://github.com/aub-mind/arabert 

Cloning into 'arabert'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 600 (delta 38), reused 45 (delta 30), pack-reused 535
Receiving objects: 100% (600/600), 9.14 MiB | 17.43 MiB/s, done.
Resolving deltas: 100% (339/339), done.


In [4]:
from transformers import BertTokenizer, AutoModelForSeq2SeqLM, pipeline
from arabert.preprocess import ArabertPreprocessor

model_name="lilouuch/mbert2mbert-arabic-text-summarization-finetuned-xsum_arabic_abstractive_final_finaln"
preprocessor = ArabertPreprocessor(model_name="")

tokenizer = BertTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
pipeline = pipeline("text2text-generation",model=model,tokenizer=tokenizer,max_length=512,truncation=True)




Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/622 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/790M [00:00<?, ?B/s]

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


In [5]:
def summerize(example):
    example['generated_text'] =pipeline(example['Document'],
            pad_token_id=tokenizer.eos_token_id,
            num_beams=3,
            repetition_penalty=3.0
            )[0]['generated_text']
    return example

In [6]:
test_ds = test_ds.map(summerize)

  0%|          | 0/4687 [00:00<?, ?ex/s]

In [9]:
test_set=test_ds.to_pandas()

In [10]:
test_set

,Document,Summary,generated_text
0,ألمانيا تواجه مصاعب في التعامل مع الأعداد المت...,كشفت ألمانيا النقاب عن خطط لإضافة الجزائر والم...,أعلنت الائتلاف الحكومي في ألمانيا عن مقترحات ل...
1,"وأكدت وسائل الإعلام السورية أن الجيش ""استعاد ا...",قال التليفزيون السوري إن قوات الحكومة استعادت ...,قال المرصد السوري لحقوق الإنسان إن قوات الحكوم...
2,وفاجأ هال سيتي الحضور بمباغتة الارسنال بهدفين ...,توج فريق الارسنال ببطولة كأس انجلترا لكرة القد...,فاز نادي أرسنال الانجليزي على ضيفه كوتشين ميون...
3,معبد يزيدي في سهول محافظة نينوى يعيش المسيحيون...,يوضع الصراع في العراق غالبا في إطار صراع بين ا...,تشهد العاصمة العراقية بغداد مؤخرا معارك عنيفة ...
4,قال التلفزيون الرسمي السوري إن القوات الحكومية...,استردت القوات الحكومية السورية ومقاتلون متحالف...,قال التلفزيون الرسمي السوري إن مسلحي تنظيم الد...
...,...,...,...
4682,رجل سوداني يغادر مخبزا وهو محمل بحقيبة مملوءة ...,مجموعة مختارة من أفضل الصور في القارة الأفريقي...,
4683,هيلاري كلينتون تتوسط محمود عباس وبنيامين نتنيا...,استأنف الرئيس الفلسطيني محمود عباس ورئيس الوزر...,قالت وزيرة الخارجية الأمريكية هيلاري كلينتون إ...
4684,ومع اتهام كارلوس غصن بمخالفات مالية عندما كان ...,هناك الكثير مما يجمع كارول نحاس بزوجها كارلوس ...,أثارت زوجة كارلوس غصن ، المرشح للانتخابات الرئ...
4685,ستفرج الحركة عن اسرى الجيش السوداني لديها ومن ...,افرجت السلطات السودانية عن 57 من معتقلي حركة ا...,اعلن مسؤولون سودانيون ان السلطات ستفرج عن سبعة...


In [11]:
test_set.to_csv('abstractive summarization arabic.csv',index=False)